In [100]:
NAME = "GROUP 21 - GROUP PROJECT"
COLLABORATORS = "WILD BUNCH"

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


# We will use matplotlib to plot figures
import matplotlib.pyplot as plt

# For regression analysis we will use the statsmodels package
import statsmodels.api as sm
from statsmodels.formula.api import ols

# For visual inspection of the regression models
from statsmodels.graphics.regressionplots import plot_regress_exog, plot_fit, plot_leverage_resid2, influence_plot
from statsmodels.discrete.discrete_model import Logit
from scipy.special import logit
import math
from scipy.stats import boxcox
from statsmodels.tools.eval_measures import rmse

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import os # accessing directory structure


In [80]:
url = "https://raw.githubusercontent.com/jpolakov/DS2_Group21_Project/main/winemag-data-130k-v2.csv"
df = pd.read_csv(url, index_col= 0)
df.head()
df.shape


In [81]:
#df.describe

In [82]:
# Find number of duplicate records
df._duplicates = df.duplicated()
print ('Number of duplicate records:{}'.format(df._duplicates.sum()))

Number of duplicate records:9983


<ipython-input-82-53e02941f89f>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df._duplicates = df.duplicated()


In [83]:
# Drop duplicates (9983 records)
df = df.drop_duplicates()
df.shape


(119988, 13)

### Checkpoint
- Folloing columns will not be used in the analysis:

- 'description','designation', 'region_2','taster_twitter_handle' 


In [84]:
# Remove Region_1 and Region_2 from the analysis
#df=df.drop(columns=['description','designation', 'region_2','taster_twitter_handle'], inplace=True)
df.drop(columns=['description','designation','region_2','taster_twitter_handle'], inplace=True)


In [85]:
df

,country,points,price,province,region_1,taster_name,title,variety,winery
0,Italy,87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,87,15.0,Douro,NaN,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,87,14.0,Oregon,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...
129966,Germany,90,28.0,Mosel,NaN,Anna Lee C. Iijima,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,US,90,75.0,Oregon,Oregon,Paul Gregutt,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,90,30.0,Alsace,Alsace,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,France,90,32.0,Alsace,Alsace,Roger Voss,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [86]:
#Find number of missing values per column
df.isnull().sum()

country           59
points             0
price           8395
province          59
region_1       19560
taster_name    24917
title              0
variety            1
winery             0
dtype: int64

### Checkpoint
- number of missing values for "country" column is matching the number of missing values for "province" column
- both are important columns used in analysis 
- we need to drop null value records

In [87]:
# MISSING DATA FIX for empty values in "country", "price" and 'variety' columns
#df.drop(df.loc[df['country'] >= 91 ].index, inplace=True)
df.dropna(subset=['country','price', 'variety'], inplace=True)

In [88]:
df.shape


(111537, 9)

In [89]:
#Find number of missing values per column
df.isnull().sum()

country            0
points             0
price              0
province           0
region_1       17957
taster_name    23272
title              0
variety            0
winery             0
dtype: int64

### Checkpoint
- number of missing values  looks better, no missing values for important columns: country, points, price, province, winery
- we are going to accept that some of the reviews does not have a name of the tester
- we do not care much about reagion for global datest, hoping there will no missing values for smaller focus datasets(US/California/etc)

### CREATING FOCUS DATASETS : 
- Focus Set 1 : US dataset (50k+ = almost 50% rows of total world dataset)
- Focus Set 2 : California (33k+ = 66% percent of US data subset)

In [90]:
# creating new US dataframe called "df_US" 
df_US = df 
df_US.shape

In [94]:
#df_US[df_US.country != 'US']
df_US.drop(df_US.loc[df_US['country'] != 'US' ].index, inplace=True)
df_US.shape

In [97]:
df_US.province.value_counts()

California           33526
Washington            7913
Oregon                4917
New York              2444
Virginia               718
Idaho                  177
Michigan               103
Texas                   89
America                 84
Colorado                62
New Mexico              44
Arizona                 38
Missouri                33
North Carolina          22
Pennsylvania            17
Ohio                    10
New Jersey               8
Washington-Oregon        7
Massachusetts            6
Illinois                 5
Iowa                     4
Connecticut              3
Nevada                   3
Vermont                  2
Rhode Island             1
Hawaii                   1
Kentucky                 1
Name: province, dtype: int64

In [98]:
# creating new CALIFORNIA dataframe called "df_US" 
df_CAL = df_US
df_CAL.drop(df_CAL.loc[df_CAL['province'] != 'California' ].index, inplace=True)
df_CAL.shape

(33526, 9)

In [99]:
df_CAL

,country,points,price,province,region_1,taster_name,title,variety,winery
10,US,87,19.0,California,Napa Valley,Virginie Boone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
12,US,87,34.0,California,Alexander Valley,Virginie Boone,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini
14,US,87,12.0,California,Central Coast,Matt Kettmann,Mirassou 2012 Chardonnay (Central Coast),Chardonnay,Mirassou
23,US,87,22.0,California,Paso Robles,Matt Kettmann,Bianchi 2011 Signature Selection Merlot (Paso ...,Merlot,Bianchi
25,US,87,69.0,California,Sonoma Coast,Virginie Boone,Castello di Amorosa 2011 King Ridge Vineyard P...,Pinot Noir,Castello di Amorosa
...,...,...,...,...,...,...,...,...,...
129942,US,90,35.0,California,Sonoma County,Virginie Boone,Arrowood 2010 Cabernet Sauvignon (Sonoma County),Cabernet Sauvignon,Arrowood
129945,US,90,20.0,California,Santa Ynez Valley,Matt Kettmann,Birichino 2013 Jurassic Park Vineyard Old Vine...,Chenin Blanc,Birichino
129949,US,90,35.0,California,Napa Valley,Virginie Boone,Flora Springs 2013 Barrel Fermented Chardonnay...,Chardonnay,Flora Springs
129950,US,90,35.0,California,Napa Valley,Virginie Boone,Hendry 2012 Blocks 7 & 22 Zinfandel (Napa Valley),Zinfandel,Hendry
